In [25]:
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# This is a swing at a custom neural network predicting soccer game results based on scores.
# I've been in Keras for a few months, but am just starting to understand my way around.
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()

# We need to calculate one team's aggregate margin in order to determine the result.
# We've chosen home for that calculation, but may as well get away as well for good measure.
# Then, the absolute margin is the number of goals between the teams.
df['h_marg'] = df['score1'] - df['score2']
df['a_marg'] = df['score2'] - df['score1']
df['margin'] = np.abs(df['score1'] - df['score2'])

results = []

for i in df['h_marg']:
  if i > 0: # If the home team's margin is greater than 0, it's a home win.
    results.append("HOME WIN")
  elif i < 0: # If the home team's margin is less than 0, it's an away win.
    results.append("AWAY WIN")
  else: # Otherwise, it's a draw.
    results.append("DRAW")

df['result'] = results

In [26]:
# The model gets a little confused the more numerical rows are fed into it, so we're keeping it simple.
X = df[['score1', 'score2', 'h_marg', 'a_marg', 'margin']].reset_index(drop=True)

# Have to factorize the labels before converting them to categorical (e.g. 1.,0.,0.).
y = df[['result']].copy()

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=6)

In [27]:
# Have to factorize the labels before converting them to categorical (e.g. 1.,0.,0.).
y_train = to_categorical(pd.factorize(y_train['result'])[0])
y_test = to_categorical(pd.factorize(y_test['result'])[0])

# The train and test data will be easier to handle in array form.
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

In [28]:
# Three layers with a 3 unit output layer given the number of outcomes (HOME WIN, AWAY WIN, DRAW).
model = Sequential()
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Definitely excess on the epochs. ¯\_(ツ)_/¯
model.fit(X_train, y_train, epochs=40, batch_size=128, validation_split=0.2, verbose=1)

Epoch 1/40
75/75 [==============================] - 1s 5ms/step - loss: 1.4946 - accuracy: 0.3116 - val_loss: 0.6594 - val_accuracy: 0.8807
Epoch 2/40
75/75 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.9375 - val_loss: 0.3077 - val_accuracy: 1.0000
Epoch 3/40
75/75 [==============================] - 0s 2ms/step - loss: 0.2527 - accuracy: 0.9997 - val_loss: 0.1569 - val_accuracy: 1.0000
Epoch 4/40
75/75 [==============================] - 0s 2ms/step - loss: 0.1290 - accuracy: 1.0000 - val_loss: 0.0816 - val_accuracy: 0.9996
Epoch 5/40
75/75 [==============================] - 0s 3ms/step - loss: 0.0641 - accuracy: 1.0000 - val_loss: 0.0412 - val_accuracy: 1.0000
Epoch 6/40
75/75 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000 - val_loss: 0.0197 - val_accuracy: 1.0000
Epoch 7/40
75/75 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 1.0000
Epoch 8/40
75/75 [==

In [30]:
# Now we will predict results based on test data.
predictions = model.predict(X_test).round(2)

In [31]:
# Reshaping y_test and predictions to be directly compared 1v1 iteratively.
predictions = predictions.reshape(23814,)
predictions = pd.Series(predictions)
predictions = pd.DataFrame(predictions).reset_index(drop=True)
y_test = y_test.reshape(23814,)
y_test = pd.Series(y_test)
y_test = pd.DataFrame(y_test).reset_index(drop=True)
PvA = pd.concat([y_test, predictions], axis=1)
PvA.columns = ["Predicted", "Actual"]

In [32]:
# If the margin is 0, there is no error. If the margin is 1, there was an error.
PvA['Margin'] = np.abs(PvA['Predicted'] - PvA['Actual'])

In [33]:
# Sum of all of the rows with errors.
PvA_err = np.sum(PvA['Margin'])

In [34]:
# Length of the prediction set.
PvA_len = len(PvA)

In [35]:
# Accuracy metric.
PvA_acc = 1-(PvA_err/PvA_len)

In [38]:
# AORTD = Accuracy on Random Test Data
print("AORTD: %.2f" % (PvA_acc * 100) + "%")

AORTD: 63.55%
